In [ ]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import preprocessing
import scipy.io
from tensorflow.keras import layers
import mne
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
from classifier_gen import EEGNet_seq, EEGNet_seq_attention
from eegnet import get_data, split_data, remove_nan, remove_participant, split_timeseries

Loading the data

In [ ]:
window_size = 2500

X, Y, order = get_data()
X, Y = split_data(X, Y, order, window_size=window_size)

length = len(X)
X = np.array(X)
X = X.reshape(length, 32, window_size, 1)
#X = np.moveaxis(X, [3], [1])
Y = np.array(Y)
#remove nan values
X, Y = remove_nan(X, Y)
X -= np.min(X)
X /= np.max(X)

#remove participant 9
X, Y, removed_participant, removed_scores = remove_participant(X, Y, order, 9)

Machin Lern